In [1]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/notebooks/..


In [2]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

urls = ('http://www.moneycontrol.com/news/business/markets/stock-radar-adani-enterprises-paytm-zomato-indigo-and-others-in-focus-12750205.html',)
tag = ('div', 'p')
tag_classes = ('content_wrapper arti-flow',)

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(tag,
            class_=tag_classes
        )
    ),
)

docs = loader.load()
doc = docs[0]

In [4]:
print(doc.page_content)

                                                                                                
 Representative image   Here's a quick look at which stocks will be in focus in trade on Tuesday, from significant investments to major deals announced over the weekend:Adani Enterprises: Gautam Adani and companies from the Adani Group acquired over a 2 percent stake in Adani Enterprises through open market purchases from September 2023 to June 2024, as reported on June 14. Prior to these acquisitions, the combined voting capital held by the acquirers and the promoter group was 71.95 percent. Following the purchases, this stake increased by 2.02 percent, bringing their total voting rights to 73.95 percent.Story continues below AdvertisementRemove AdPaytm & Zomato: Paytm, also known as One97 Communications Ltd., is reportedly in advanced discussions with Zomato Ltd. to sell its movie and events ticketing business. This move is part of Paytm's strategy to revamp amid declining sales. While ta

In [5]:
from pydantic import BaseModel, Field
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [6]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [7]:
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            )

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

/Users/L024258/lilly_work/github-copilot/exploration/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [9]:
tagging_chain.invoke({"input": doc})

{'summary': 'This article discusses various stocks that will be in focus in trade on Tuesday. It includes information about Adani Enterprises, Paytm & Zomato, Zydus, Brightcom Group, Biocon, Jubilant Pharma, TCS, Lodha, Purvankara, Bharti Airtel, Indus Tower, Pidilite Industries, Central Bank of India, LIC, and SpiceJet & IndiGo. The article provides updates on significant investments, major deals, and other relevant news related to these companies.',
 'language': 'English',
 'keywords': 'stocks, trade, Adani Enterprises, Paytm, Zomato, Zydus, Brightcom Group, Biocon, Jubilant Pharma, TCS, Lodha, Purvankara, Bharti Airtel, Indus Tower, Pidilite Industries, Central Bank of India, LIC, SpiceJet, IndiGo'}

In [10]:
from typing import List

class Tip(BaseModel):
    """Information about papers mentioned."""
    company_name: str = Field(description="The name of the company.")
    sentiment: str = Field(description="The sentiment associated with the company in the text.")
    signal: str = Field(description="Infer BUY or SELL signal associated with the company in the text.")
    signal_phrase: str = Field(description="The signal associated with the company in the text.")


class Info(BaseModel):
    """Information to extract"""
    tips: List[Tip]

In [ ]:
template = """A news article will be passed to you. Extract from it all the company that are mentioned in this article. 
If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.
Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [13]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

tip_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=tip_extraction_function, 
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="tips")

In [14]:
extraction_chain.invoke({"input": doc})

[{'company_name': 'Adani Enterprises',
  'sentiment': 'Positive',
  'signal': 'Acquisition'},
 {'company_name': 'Paytm & Zomato',
  'sentiment': 'Neutral',
  'signal': 'Potential Sale'},
 {'company_name': 'Zydus Lifesciences Ltd',
  'sentiment': 'Negative',
  'signal': 'FDA Classification'},
 {'company_name': 'Brightcom Group',
  'sentiment': 'Neutral',
  'signal': 'Announcement of Results'},
 {'company_name': 'Biocon',
  'sentiment': 'Neutral',
  'signal': 'FDA Observations'},
 {'company_name': 'Jubilant Pharma',
  'sentiment': 'Positive',
  'signal': 'Investment'},
 {'company_name': 'Tata Consultancy Services',
  'sentiment': 'Negative',
  'signal': 'Adverse Judgment'},
 {'company_name': 'Lodha',
  'sentiment': 'Neutral',
  'signal': 'Final Order on Stamp Duty'},
 {'company_name': 'Purvankara',
  'sentiment': 'Positive',
  'signal': 'Land Acquisition'},
 {'company_name': 'Bharti Airtel Ltd.',
  'sentiment': 'Positive',
  'signal': 'Prepayment of Liabilities'},
 {'company_name': 'Indu